In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *
DATA_DIR = '/work/users/k/4/k4thryn/Repos/OLD_EpSampling_Nov2024/data/'

# <font color=blue> 1) Dataset

In [ ]:
ts = '20241108-123515'
fpath = os.path.join(DATA_DIR,'processed',f'training_target_df_{ts}.csv')
dff = pd.read_csv(fpath)

pd.set_option('display.max_columns', None)

print(dff.columns)
display(dff)
d = {'Proj_state_inc_deaths':'PROJ state deaths',
     'True_county_inc_deaths':'TRUE county deaths',
     'Pop':'County population',
     'Naive_proj_deaths':'Naive PROJ deaths',
     '':'',
    }

dff.rename(d,axis=1,inplace=True)

In [ ]:
Y_COL = 'TRUE county deaths'
X_COLS = ['PROJ state deaths','County population']
NAIVE_COL = 'Naive PROJ deaths'

dff.dropna(inplace=True)
df = dff[dff[Y_COL] >= 0]

def get_model_df(df, X_COLS, Y_COL, NAIVE_COL):
    cols = ['Date', 'Fips'] + X_COLS + [Y_COL, NAIVE_COL]
    df = df[cols]    
    return df

df = get_model_df(df, X_COLS, Y_COL, NAIVE_COL)
display(df)


import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1, style='ticks', palette='muted') 
DPI = 80
FIGSIZE = (8,4)

# plt.figure(figsize=FIGSIZE)
# sns.histplot(dff[Y_COL].values, bins=40, kde=True, element='step')
# sns.despine()
# # sns.kdeplotdf[TARGET_COL].values)
# plt.xlim((-20, 90))
# plt.gcf().set_dpi(DPI)
# plt.show()

plt.figure(figsize=FIGSIZE)
sns.histplot(df[Y_COL].values, bins=40, kde=True, element='step')
sns.despine()
# sns.kdeplotdf[TARGET_COL].values)
plt.xlim((-20, 90))
plt.gcf().set_dpi(DPI)
plt.show()

print(f"Are all targets non-negative? {(df[Y_COL].min() >= 0)}")

## <font color=blue> 2) Training: County population + Predicted state deaths

    
<font color=blue> _Compare regression algorithms:_
1. <font color=blue>  Linear 
1. <font color=blue>  Poisson 
1. <font color=blue>  Zero-inflated
    
This is our most simplistic model. We are predicting the number of county deaths given two covariates: 1) projected state deaths (from COVIDHub) and 2) county population. We will consider vanilla linear regression, poisson regression, and zero-inflated regression.
    
    
### <font color=red> Add histogram of target values?

### <font color=blue> Model design
- <font color=blue> **_Train-test split:_** Our training and test/evaluation set will be partitioned based on (2 month)-wise chunks. That is, we will evaluate our model on temporally consecutive targets over the course of 8 weeks, and we will train on the remaining weeks.
    - <font color=black> **Train samps:** 7939
    - <font color=black> **Test samps:** 771
- <font color=blue> **_Cross validation protocol:_** We will train a model for each month, for a total of 10 models. From these, we can evaluate average performance for each regression algorithm.
    

In [ ]:
from epsampling.utils import get_chunks
from epsampling.modeling import get_date_chunked_splits, get_performance
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [ ]:
def get_df_res(df_train, df_test, X_COLS, Y_COL, PRED_COL, ALG):    
    X_train = df_train[X_COLS]
    y_train = df_train[Y_COL]
    X_test = df_test[X_COLS]
    
    df_pred = df_test.copy()
    reg = linear_model.LinearRegression().fit(X_train, y_train)
    df_pred['Algorithm'] = ALG
    df_pred[PRED_COL] = reg.predict(X_test)
    
    return df_pred



# idx = 5
# i = 0 ## cross-val progress
ALG = 'Linear Regression'


res_dfs = []
models = {}

chunks = get_chunks(list(df.Date.unique()), num_membs=8)

for i,test_chunk in enumerate(chunks):
#     print(i,test_chunk)
    
    df_train, df_test = get_date_chunked_splits(df, chunks, i)
    df_test[f'Run'] = i
    
    PRED_COL = 'PRED county deaths'
    df_res = get_df_res(df_train, df_test, X_COLS, Y_COL, PRED_COL, ALG)
    
    if i==0: display(df_test.head(), df_res.head())
        
    res_dfs.append(df_res)
    
df_full_res = pd.concat(res_dfs)
# df_full_res
    
    
# df_train, df_test = get_date_chunked_splits(df, chunks, idx)
# # print(f'Training set size: {len(df_train)}\nTest set size: {len(df_test)}')
# df_test[f'Run'] = i


# for i,idx in tqdm(enumerate(idc), total=len(idc)):


# pred_col = 'PRED county inc. deaths'
# df_res = get_df_res(df_train, df_test, X_COLS, Y_COL, pred_col)
# df_res.head()

## <font color=blue> 3) Evaluation

## <font color=blue> a. _Metrics_ 
Average across cross-validation splits.

In [ ]:
## GET PERFORMANCE !
# from epsampling.modeling import get_metrics_ser

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error

def get_metrics_ser(df, target_col, pred_col, alg_col, naive_col):
    
    metrics_dict = {'MAE': mean_absolute_error,
                    'MedAE': median_absolute_error,
                    'MSE': mean_squared_error,
                    'RMSE': mean_squared_error,
                    'r2': r2_score
                    }
    
    mae = mean_absolute_error(df[target_col], df[pred_col])
    medae = median_absolute_error(df[target_col], df[pred_col])
    r2 = r2_score(df[target_col], df[pred_col])
    mse = mean_squared_error(df[target_col], df[pred_col])
    
    relmae = mae / mean_absolute_error(df[target_col],df[naive_col])
    
    ser = {'Algorithm':alg_col, 'MAE':mae, 'MedAE':medae, 
           'R-squared':r2, 'MSE': mse, 'relMAE': relmae,}
    
    return ser
    

metric_sers = []

for run in df_full_res.Run.unique():
    
    subdf = df_full_res[df_full_res['Run']==run]
    ser = get_metrics_ser(subdf, 
                          target_col=Y_COL,
                          pred_col=PRED_COL,
                          alg_col=ALG,
                          naive_col=NAIVE_COL)
    
    metric_sers.append(ser)
    
df_run = pd.DataFrame(metric_sers)
df_run

In [ ]:
list_run_res = []


In [ ]:
df_full_res

In [ ]:
df_melt = pd.melt(df_run, id_vars=['Algorithm'], 
                  value_vars=['MAE', 'MedAE', 'R-squared','MSE','relMAE'], 
                  var_name='Metric', value_name='Score')

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(font_scale=1, style='whitegrid', palette='pastel') 

# DPI = 80

# for metric in df_melt.Metric.unique():
#     df_plot = df_melt[df_melt.Metric==metric]
# #     display(df_plot)
#     sns.boxplot(df_plot, x='Algorithm',y='Score')   
#     plt.title(f'{metric}')
#     plt.xticks(rotation=45,ha='right')
#     plt.gcf().set_dpi(DPI)
#     plt.show()

# df_full_res

for metric in  ['MAE', 'MedAE', 'R-squared', 'MSE'    ,'relMAE']:
    dff_melt = df_melt[df_melt['Metric']==metric]
    print(f'\n* {metric} *  \n Mean: {round(dff_melt.Score.mean(),3)}\n Median: {round(dff_melt.Score.median(),3)}')

## <font color=blue> b. _Plots_

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1, style='ticks', palette='muted') 

DPI = 80
FIGSIZE = (8,4)

df = df_pred

plt.figure(figsize=FIGSIZE)
sns.scatterplot(data=df, x=TARGET_COL, y=pred_col, alpha=alpha, sizes=sizes, linewidth=linewidth)
sns.despine()
plt.xlim((-10, 90))
plt.gcf().set_dpi(DPI)
plt.show()



figsize = (8,6)
alpha = 0.15
sizes = 2
linewidth = 0

sns.regplot(data=df,  x=TARGET_COL, y=pred_col,
            scatter_kws={'s':10, 'alpha':0.5}, line_kws={'color':'k','linewidth':1})
sns.despine()
# plt.title('raw') 
# plt.title('y+(1)/pop*100000\n r2: 0.934336') 
# plt.title('y/pop*100000\n r2: -0.016382') 

plt.show()

# <font color=red> What about when transformed?